# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 29 2022 2:37PM,240773,19,ADV80003612,"AdvaGen Pharma, Ltd",Released
1,Jun 29 2022 2:25PM,240772,10,DNMD-22737,Emerginnova,Released
2,Jun 29 2022 2:25PM,240771,20,ALUR038194534,Alumier Labs Inc.,Released
3,Jun 29 2022 2:25PM,240771,20,ALUR247275603,Alumier Labs Inc.,Released
4,Jun 29 2022 2:25PM,240771,20,ALUR339178640,Alumier Labs Inc.,Released
5,Jun 29 2022 2:25PM,240771,20,ALUR428169294,Alumier Labs Inc.,Released
6,Jun 29 2022 2:25PM,240771,20,ALUR403793311,Alumier Labs Inc.,Released
7,Jun 29 2022 2:25PM,240771,20,ALUR273573234,Alumier Labs Inc.,Released
8,Jun 29 2022 2:20PM,240770,10,SONSB0001848,"Nextsource Biotechnology, LLC",Released
9,Jun 29 2022 2:02PM,240769,19,ACG-2102490486,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,240769,Released,4
33,240770,Released,1
34,240771,Released,6
35,240772,Released,1
36,240773,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240769,NaN,NaN,4.0
240770,NaN,NaN,1.0
240771,NaN,NaN,6.0
240772,NaN,NaN,1.0
240773,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240662,0.0,0.0,1.0
240665,0.0,0.0,1.0
240671,0.0,0.0,1.0
240672,11.0,7.0,14.0
240679,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240662,0,0,1
240665,0,0,1
240671,0,0,1
240672,11,7,14
240679,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240662,0,0,1
1,240665,0,0,1
2,240671,0,0,1
3,240672,11,7,14
4,240679,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240662,,,1
1,240665,,,1
2,240671,,,1
3,240672,11,7,14
4,240679,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd"
1,Jun 29 2022 2:25PM,240772,10,Emerginnova
2,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.
8,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC"
9,Jun 29 2022 2:02PM,240769,19,ACG North America LLC
13,Jun 29 2022 1:53PM,240768,17,"Nextsource Biotechnology, LLC"
14,Jun 29 2022 1:51PM,240767,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
17,Jun 29 2022 1:47PM,240766,10,"Nextsource Biotechnology, LLC"
18,Jun 29 2022 1:11PM,240763,20,Alumier Labs Inc.
24,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",,,1
1,Jun 29 2022 2:25PM,240772,10,Emerginnova,,,1
2,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,,,6
3,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",,,1
4,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,,,4
5,Jun 29 2022 1:53PM,240768,17,"Nextsource Biotechnology, LLC",,,1
6,Jun 29 2022 1:51PM,240767,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,3
7,Jun 29 2022 1:47PM,240766,10,"Nextsource Biotechnology, LLC",,,1
8,Jun 29 2022 1:11PM,240763,20,Alumier Labs Inc.,,,6
9,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 29 2022 2:25PM,240772,10,Emerginnova,1,,
2,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,6,,
3,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",1,,
4,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,4,,
5,Jun 29 2022 1:53PM,240768,17,"Nextsource Biotechnology, LLC",1,,
6,Jun 29 2022 1:51PM,240767,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
7,Jun 29 2022 1:47PM,240766,10,"Nextsource Biotechnology, LLC",1,,
8,Jun 29 2022 1:11PM,240763,20,Alumier Labs Inc.,6,,
9,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 29 2022 2:25PM,240772,10,Emerginnova,1,,
2,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,6,,
3,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",1,,
4,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jun 29 2022 2:25PM,240772,10,Emerginnova,1.0,NaN,NaN
2,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,6.0,NaN,NaN
3,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jun 29 2022 2:25PM,240772,10,Emerginnova,1.0,0.0,0.0
2,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,6.0,0.0,0.0
3,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2648049,35.0,12.0,0.0
12,481389,1.0,1.0,1.0
17,240768,1.0,0.0,0.0
18,240662,1.0,0.0,0.0
19,1444415,26.0,7.0,11.0
20,1444526,83.0,10.0,0.0
21,240679,1.0,0.0,0.0
22,722150,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2648049,35.0,12.0,0.0
1,12,481389,1.0,1.0,1.0
2,17,240768,1.0,0.0,0.0
3,18,240662,1.0,0.0,0.0
4,19,1444415,26.0,7.0,11.0
5,20,1444526,83.0,10.0,0.0
6,21,240679,1.0,0.0,0.0
7,22,722150,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35.0,12.0,0.0
1,12,1.0,1.0,1.0
2,17,1.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,26.0,7.0,11.0
5,20,83.0,10.0,0.0
6,21,1.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,12,Released,1.0
2,17,Released,1.0
3,18,Released,1.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,17,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,1.0,0.0,0.0,11.0,0.0,0.0,0.0
Executing,12.0,1.0,0.0,0.0,7.0,10.0,0.0,0.0
Released,35.0,1.0,1.0,1.0,26.0,83.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,17,18,19,20,21,22
0,Completed,0.0,1.0,0.0,0.0,11.0,0.0,0.0,0.0
1,Executing,12.0,1.0,0.0,0.0,7.0,10.0,0.0,0.0
2,Released,35.0,1.0,1.0,1.0,26.0,83.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,17,18,19,20,21,22
0,Completed,0.0,1.0,0.0,0.0,11.0,0.0,0.0,0.0
1,Executing,12.0,1.0,0.0,0.0,7.0,10.0,0.0,0.0
2,Released,35.0,1.0,1.0,1.0,26.0,83.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()